# Mosaic from a single multitemporal dataset


The goal of this notebook is to provide an example of how to create a cloud-free mosaic from Sentinel-2 imagery over a specific area over a time period. We first use `satsearch` to search for Sentinel-2 data then combine them together using `stackstac`. A median operation will be applied to merge the images into a single layer that could be save off into Azure blob storage as COGs for later use.


## 1. Sentinel-2 Dataset

Satellite images (also Earth observation imagery, spaceborne photography, or simply satellite photo) are images of Earth collected by imaging satellites operated by governments and businesses around the world (see https://en.wikipedia.org/wiki/Satellite_imagery). Its major applications include Earth observation and land cover monitoring. 


SENTINEL-2 (https://sentinel.esa.int/web/sentinel/user-guides/sentinel-2-msi/overview) is a wide-swath, high-resolution, multi-spectral imaging mission, supporting Copernicus Land Monitoring studies, including the monitoring of vegetation, soil and water cover, as well as observation of inland waterways and coastal areas.

## 2. Imports

In [ ]:
import stackstac
from satsearch import Search

import xrspatial.multispectral as ms

import matplotlib.pyplot as plt

## 3. Load Sentinel 2 data

In this example, we use data from `sentinel-s2-l2a-cogs` collection within a bounding box of `[-93.112301, 29.649001, -92.075965, 30.719868]`, and the time range considered is from `2019-07-01` to `2020-06-30`. And the collected data has less than 25% cloud coverage.

In [ ]:
items = Search(
    url="https://earth-search.aws.element84.com/v0",
    bbox=[-93.112301, 29.649001, -92.075965, 30.719868],
    collections=["sentinel-s2-l2a-cogs"],
    query={'eo:cloud_cover': {'lt': 25}},
    datetime="2019-07-01/2020-06-30"
).items()

len(items)

Let's combine all the above STAC items into a lazy xarray with following settings:
- projection: epsg=32613
- resolution: 100m
- bands: green (B02), red (B03), blue (B04), NIR (B08), SWIR1 (B11)

In [ ]:
bands = ['B02', 'B03', 'B04', 'B08', 'B11']

stack_ds = stackstac.stack(
    items, epsg=32613, resolution=100, assets=bands
)

stack_ds

We can get a median composite for each month in the considered period of time:

In [ ]:
monthly = stack_ds.resample(time="MS").median("time", keep_attrs=True)
monthly

In [ ]:
import dask.diagnostics
with dask.diagnostics.ProgressBar():
    monthly = monthly.compute()

## 4. Cloud-free scene using median operator

In this step, we use a median operation to merge all monthly images into 1 single cloud-free layer. With an assumption that, along a multitemporal stack, clouds would not persist at the same geographical position from time to time (i.e image to image), the more data we have, the higher chance of dropping clouds.

In [ ]:
median_scene = monthly.median(dim=['time'])

With 3 bands: red, green, blue, let's see the true color using the `true_color` function from `xrspatial.multispectral module` for each separate month and the median layer.

In [ ]:
bands_mapping = {v: i for i, v in enumerate(bands)}

band_blue = bands_mapping['B02']
band_green = bands_mapping['B03']
band_red = bands_mapping['B04']

In [ ]:
months = 12
imgs = []
for month in range(months):
    # True color
    r = monthly[month][band_red]
    g = monthly[month][band_green]
    b = monthly[month][band_blue]
    img = ms.true_color(r, g, b)
    imgs.append(img)

In [ ]:
# Utility function for displaying images

def display_images(images, columns=2, width=50, height=50):
    height = max(height, int(len(images)/columns) * height)
    plt.figure(figsize=(width, height))
    for i, image in enumerate(images):
        plt.subplot(len(images) / columns + 1, columns, i + 1)
        plt.imshow(image)

#### Monthly data

In [ ]:
# takes some time to run
display_images(imgs)

#### Median layer

In [ ]:
median_scene = monthly.median(dim=['time'])

median_red_agg = median_scene[band_red]
median_green_agg = median_scene[band_green]
median_blue_agg = median_scene[band_blue]

median_img = ms.true_color(median_red_agg, median_green_agg, median_blue_agg)

median_img

### References

- https://stackstac.readthedocs.io/en/latest/basic.html